In [1]:
import ee
import geemap

In [2]:
ee.Initialize()
Map = geemap.Map(center = [28.3949, 84.1240], zoom = 7)
Map

Map(center=[28.3949, 84.124], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [3]:
Np = ee.FeatureCollection("users/aaditya-rb/nepal_map")
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
DEM = ee.Image("USGS/SRTMGL1_003")
L7_vis = {"opacity":1,"bands":["B4","B3","B2"],"min":220,"max":19660,"gamma":1}

L7_np = L7.filterDate('2020-01-01', '2020-12-30') \
.filterBounds(Np) \
.filterMetadata("CLOUD_COVER", 'less_than', 10) \
.sort('CLOUD_COVER',False)

In [4]:
mergedL7 = L7_np.median().clip(Np)

# Map.addLayer(mergedL7, L7_vis, "Landsat 7")
# Map.addLayer(LULC_icimod, LULC_vis, "LULC icimod")

b2 = mergedL7.select("B2")
b3 = mergedL7.select("B3")
b4 = mergedL7.select("B4")
b5 = mergedL7.select("B5")
NDVI = b4.subtract(b3).divide(b4.add(b3)).rename("NDVI")
NDBI = b5.subtract(b4).divide(b5.add(b4)).rename("NDBI")
NDWI = b4.subtract(b5).divide(b5.add(b4)).rename("NDWI")
MNDWI = b2.subtract(b5).divide(b2.add(b5)).rename("MNDWI")
DEM_nepal = DEM.clip(Np)

In [5]:
addedNepal = mergedL7

In [6]:
exportBands = mergedL7.select("B1").addBands(mergedL7.select("B2")) \
.addBands(mergedL7.select("B3")) \
.addBands(mergedL7.select("B4")) \
.addBands(mergedL7.select("B5")) \
.addBands(mergedL7.select("B6")) \
.addBands(mergedL7.select("B7")) \
.addBands(NDVI.select("NDVI")) \
.addBands(NDBI.select("NDBI")) \
.addBands(NDWI.select("NDWI"))\
.addBands(MNDWI.select("MNDWI"))\
.addBands(DEM_nepal) \
# .addBands(LULC_icimod.select("b1").rename("LULC"))

In [8]:
# MAKE SURE ASSET OF RANDOM FOREST <10.4MB is uploaded

In [9]:
from geemap import ml

In [10]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection("users/aaditya-rb/random_forest_strings_equal_class")

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

In [11]:
# feature names
feature_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDWI', 'NDVI', 'MNDWI', 'NDBI', 'elevation']

# classify the image again but with the classifier from the persisted trees
classified = exportBands.select(feature_names).classify(another_classifier)

In [12]:
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

In [13]:
Map

Map(bottom=13987.0, center=[28.3949, 84.124], controls=(WidgetControl(options=['position', 'transparent_bg'], …

In [14]:
geemap.ee_export_image_to_drive(classified, description='LULC_MAP_2020_COMP_proper_CLASS', folder='GEE export folder', region=Np.geometry(), file_format='GeoTIFF', format_options={'cloudOptimized': True}, scale=30)


Exporting LULC_MAP_2020_COMP_proper_CLASS ...


In [7]:
# geemap.ee_export_image(classified, filename="LULC_MAP_2020.tif", region=Np.geometry(), scale=500)
